# CS579: Online Social Network Analysis

# Final Project - Recommendation Systems Using Map-Reduce


$$J V P S Avinash $$ <br>
$$Rakshith Muniraju $$



# Part - 1 : Data Generation

This file contains the data generation code to extract the data from Rotten Tomatoes Movie Data Base. <br>
The data generation contains two steps: <br>
1) Connect to Rotten Tomatoes API to get the top 100 movies from the last two years. <br>
2) Implement a Web Crawler to extract the User-Rating information from the website.

In [1]:
from BeautifulSoup import BeautifulSoup
import sys
import re
import urllib
import urllib2
import urlparse
import io
import rtsimple as rt
from unidecode import unidecode
import subprocess

In [2]:
class MyOpener(urllib.FancyURLopener):
    version = 'Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2.15) Gecko/20110303 Firefox/3.6.15'
    Connection= 'keep-alive'

In [3]:
rt.API_KEY = 'usdusq9ana3aq2r637nw5572'

My Opener function helps in connecting to the Firefox to implemet Web Crawler

In [ ]:
ALL_URLs_TOP_RT_2015 = list()
ALL_URLs_TOP_RT_2014 = list()
myopener = MyOpener()

The following couple cells of codes contains the information of how to connect to the Rotten Tomatoes site to 
extract the information of top 100 movies from the years 2015 and 2014.

In [ ]:
PAGE_URL_TOP_RT = "http://www.rottentomatoes.com/top/bestofrt/?year=2015"
soup = BeautifulSoup(myopener.open(PAGE_URL_TOP_RT).read())
if soup.find('title').contents[0] == "Page Not Found":
    print "Not Found"
else:
    all_movies_rt = soup.findAll('table',{'class':'table'})
    all_movies_rt2 = all_movies_rt[0].findChildren('a',{'class':'unstyled articleLink'})
for i3 in range(0,len(all_movies_rt2)):
    ALL_URLs_TOP_RT_2015.append('http://www.rottentomatoes.com'+all_movies_rt2[i3]['href'])

In [ ]:
PAGE_URL_TOP_RT = "http://www.rottentomatoes.com/top/bestofrt/?year=2014"
soup = BeautifulSoup(myopener.open(PAGE_URL_TOP_RT).read())
if soup.find('title').contents[0] == "Page Not Found":
    print "Not Found"
else:
    all_movies_rt = soup.findAll('table',{'class':'table'})
    all_movies_rt2 = all_movies_rt[0].findChildren('a',{'class':'unstyled articleLink'})
for i3 in range(0,len(all_movies_rt2)):
    ALL_URLs_TOP_RT_2014.append('http://www.rottentomatoes.com'+all_movies_rt2[i3]['href'])

In [ ]:
SUB_URL   = '/reviews/?page='
END_URL   = "&type=user&sort="


Since the API uses stars for ratings and a number of 0.5 for partial ratings, we need to write a function to
calculate rating for those users.

In [ ]:
def calculate_rating(rating):
    number_of_ratings = len(rating) - 1 #5
    full_ratings = rating.findAll('span') #4
    half_ratings = number_of_ratings - len(full_ratings) #1
    return len(full_ratings) + (half_ratings*0.5)

The below cell code crawls over the web site to extract the information about the ratings given by user in 2015.

In [ ]:
fo = open('output_v2', 'w')
count = 0
for movie_urls in ALL_URLs_TOP_RT_2015:
    for page_number in range(1,6):
        URL = movie_urls + SUB_URL + str(page_number) + END_URL
        print URL
        
        soup = BeautifulSoup(myopener.open(URL).read())
        if soup.find('title').contents[0] == "Page Not Found":
            print "Not Found"
        all_users = soup.findAll('a', {'class':'bold unstyled articleLink'})
        all_ratings_div = soup.findChildren('div',{'class':'col-xs-16'})
        print len(all_users),len(all_ratings_div)
        
        for user in range(0,len(all_ratings_div)):
            count = count + 1
            all_ratings = all_ratings_div[user].findChildren('span', {'class':'fl'})
            if len(all_ratings) == 0:
                fo.write('|'.join([unidecode(all_users[user].contents[0]), movie_urls[32:len(movie_urls)-1] , str(0)]) + '\n')
            else:
                rating = calculate_rating(all_ratings[0])
                if len(all_users[user].contents) == 0:
                    fo.write('|'.join(["rt_user_anonymus_"+str(count), movie_urls[32:len(movie_urls)-1] , str(rating)]) + '\n')
                else:
                    #print all_users[user].contents[0]
                    fo.write('|'.join([unidecode(all_users[user].contents[0]), movie_urls[32:len(movie_urls)-1] , str(rating)]) + '\n')

The below cell code crawls over the web site to extract the information about the ratings given by user in 2014.

In [ ]:
fo = open('output_v3', 'w')
count = 0
for movie_urls in ALL_URLs_TOP_RT_2014:
    for page_number in range(1,6):
        URL = movie_urls + SUB_URL + str(page_number) + END_URL
        print URL
        
        soup = BeautifulSoup(myopener.open(URL).read())
        if soup.find('title').contents[0] == "Page Not Found":
            print "Not Found"
        all_users = soup.findAll('a', {'class':'bold unstyled articleLink'})
        all_ratings_div = soup.findChildren('div',{'class':'col-xs-16'})
        print len(all_users),len(all_ratings_div)
        
        for user in range(0,len(all_ratings_div)):
            count = count + 1
            all_ratings = all_ratings_div[user].findChildren('span', {'class':'fl'})
            if len(all_ratings) == 0:
                fo.write('|'.join([unidecode(all_users[user].contents[0]), movie_urls[32:len(movie_urls)-1] , str(0)]) + '\n')
            else:
                rating = calculate_rating(all_ratings[0])
                if len(all_users[user].contents) == 0:
                    fo.write('|'.join(["rt_user_anonymus_"+str(count), movie_urls[32:len(movie_urls)-1] , str(rating)]) + '\n')
                else:
                    #print all_users[user].contents[0]
                    fo.write('|'.join([unidecode(all_users[user].contents[0]), movie_urls[32:len(movie_urls)-1] , str(rating)]) + '\n')

We merge two files into a single file by opening the shell of python.

In [ ]:
subprocess.Popen('cat output_v2 output_v3 > ratings_data',shell=True)

In [4]:
!head ratings_data

Kevin M. W|mad_max_fury_road|4.5
familiar s|mad_max_fury_road|2.5
Phil H|mad_max_fury_road|3.5
Wildaly M|mad_max_fury_road|4.0
Kase V|mad_max_fury_road|5.0
RCCLBC|mad_max_fury_road|4.0
Chris G|mad_max_fury_road|5.0
Al S|mad_max_fury_road|5.0
Carlos M|mad_max_fury_road|4.5
Eugene B|mad_max_fury_road|4.5


In [5]:
!wc -l ratings_data

14374 ratings_data


For the demo, we are using the sample file which is randomly downloaded over the net.


In [6]:
!head samp.csv

Jack Matthews|Lady in the Water|3.000000
Jack Matthews|Snakes on a Plane|4.000000
Jack Matthews|You, Me and Dupree|3.500000
Jack Matthews|Superman Returns|5.000000
Jack Matthews|The Night Listener|3.000000
Mick LaSalle|Lady in the Water|3.000000
Mick LaSalle|Snakes on a Plane|4.000000
Mick LaSalle|Just My Luck|2.000000
Mick LaSalle|Superman Returns|3.000000
Mick LaSalle|You, Me and Dupree|2.000000


In [7]:
!wc -l samp.csv

35 samp.csv
